In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
%matplotlib inline
warnings.filterwarnings(action='ignore')

dataset = pd.read_csv('../input/netflix-shows/netflix_titles.csv')

print(dataset.info())
print(dataset.isnull().sum())

# 결측치 처리 

'director'는 결측치 비율이 높다. 선택에 따라서 삭제하고 진행할 수 있지만 영화나 드라마의 매니아층은 director에 따라 콘텐츠를 선택하는 경우도 있으므로 최대한 데이터를 활용할 수 있도록 결측치를 알 수 없음으로 채웠다. 'cast'도 위와 마찬가지의 이유로 같은 방식으로 결측치를 처리하였다.

# Missing Values

'director' has a high missing value ratio. It can be deleted by choice, but movie and drama enthusiasts often choose content according to the director. Therefore, we filled the missing value with 'unknown' so that we can make the most of the data. 


'cast' treated missing values in the same way for the same reasons as above.

In [ ]:
dataset['director'].fillna('unknown', inplace = True)
dataset['cast'].fillna('unknown', inplace = True)

'country'는 약 6.5%의 결측비율을 가진다. 'director'나 'cast'의 데이터가 존재하고 두 컬럼에 나타난 인물의 국적 등을 파악할 수 있다면 제작된 국가를 유추할 수 있다. 하지만 이러한 정보가 없기 때문에 최빈값으로 결측치를 채웠다.

'country' has a missing ratio of about 6.5 per cent. 
If data from "director" or "cast" exists and the nationality of the person can be identified, it can be inferred from the country of production.
However, due to the lack of such information, the missing value was filled with the lowest value.

In [ ]:
x = dataset['country'].value_counts().head(10)
g = sns.barplot(x = x.values, y = x.index)

dataset['country'].fillna('United States', inplace = True)

'release_year'는 콘텐츠의 출시년도를 나타내고 'date_added'는 콘텐츠를 넷플릭스에서 수급한 날짜이다, 출시를 넷플릭스에서 진행했거나 수급한 년도가 출시된 년도와 동일하다고 가정하고 결측치를 채웠다.

'release_year' is the release year of the content.
'date_added' is the date when content was received from Netflix. 
Since the two columns had the same year most often, 'release_year' was used to fill the missing values of 'date_added'.

In [ ]:
dataset['year_added'] = dataset['date_added'].str.split(', ').str[1]
dataset = dataset.drop('date_added', axis = 1)

index_NaN_year = list(dataset[dataset['year_added'].isnull()].index)
dataset['year_added'].iloc[index_NaN_year] = dataset['release_year'].iloc[index_NaN_year]
dataset['year_added'] = dataset['year_added'].astype('int')

'rating'는 최빈값을 이용하여 결측치를 채웠다.

'rating' used the most frequent values to fill the missing values.

In [ ]:
x = dataset['rating'].value_counts()
g = sns.barplot(x = x.values, y = x.index)
dataset['rating'].fillna('TV-MA', inplace = True)

# 데이터 분석
먼저, 넷플릭스에서 영화와 드라마의 수급비율을 알아보았다. 드라마에 비해 영화 콘텐츠의 비율이 높았다.

# Data Analysis

First, I checked the supply-demand ratio of movies and dramas on Netflix. The proportion of movie content was higher than that of dramas.

In [ ]:
wedgeprops = {'width': 0.7, 'edgecolor': 'w', 'linewidth': 5}
plt.pie(x = dataset['type'].value_counts(), labels = dataset['type'].value_counts().index, autopct = '%.1f%%', startangle = 260, counterclock = False, wedgeprops = wedgeprops)

'supply_year' 컬럼은 넷플릭스에서 영화나 드라마를 수급한 년도에서 실제 출시년도를 빼서 수급기간을 알아보았다. 넷플릭스는 영화나 드라마에 제작을 맡고 넷플릭스에서만 해당 콘텐츠를 최초 공개 및 방영할 수 있는 넷플릭스 오리지널을 진행하고 있어, 수급기간이 거의 없는 경우가 가장 많았다. 또한, 넷플릭스 오리지널이 아니더라고 수급기간이 0 ~ 1년의 기간동안 절반이 넘는 작품을 수급하는 것으로 보아, 최신작을 빠르게 수급하려는 노력이 보인다. 넷플릭스에서는 최신작이 아닌 콘텐츠에 대해서도 일정 비율 수급하고 있는 것을 알 수 있다.

I defined 'supply_year' to see how fast Netflix receives movies and dramas. 
Netflix is trying to quickly supply and demand the latest works within a year of the release of the content. 
Netflix's participation in production and its exclusive release on the platform also seems to have had a big impact.
You can see that Netflix also supplies a certain percentage of content that is not the latest.


In [ ]:
dataset['supply_year'] = dataset['year_added'] - dataset['release_year']
x = dataset['supply_year'].value_counts().head(10)
g = sns.barplot(y = x.values, x = x.index)

다음은 넷플릭스에서 최근 5년간 콘텐츠 동향을 파악하고자 한다.

Next, we want to understand the trend of content on Netflix over the past five years.

In [ ]:
dataset_last_5 = pd.concat(pd.DataFrame(dataset[dataset['year_added'] == (2020 - i)]) for i in range(5))

넷플릭스는 다양한 사용자들이 취향에 맞게 콘텐츠를 활용할 수 있도록 많은 콘텐츠를 수급비율을 늘리고 있다. OTT 서비스이기 때문에 수급되는 양이 많을수록 회사의 성장률을 간접적으로 파악할 수 있다.

Netflix is increasing the supply and demand ratio of a lot of content so that various users can use it according to their tastes. 
Because it is an OTT service, the larger the amount supplied, the indirectly identifies the company's growth rate.

In [ ]:
x = dataset_last_5['year_added'].value_counts()
plt.pie(x = dataset_last_5['year_added'].value_counts(), labels = dataset_last_5['year_added'].value_counts().index, autopct = '%.1f%%', startangle = 260, counterclock = False, wedgeprops = wedgeprops)

아래의 그래프는 최근 5년동안 넷플릭스가 'Movie'와 'TV Show'의 수급량을 나타내는 그래프이다. 넷플릭스는 2016년에 비해 2017년에는 콘텐츠의 수급량을 큰 폭으로 늘렸으며, 2020년 영화 수급이 조금 줄어든 것을 제외하면 대부분 수급량을 늘려 다양한 볼거리를 제공하고 있다.

영화 콘텐츠의 성장비율이 높았지만 2020년에는 성장율이 소폭 감소하였다. 'TV Show'는 영화 콘텐츠에 비해 성장율이 크진 않지만 꾸준히 증가추세를 보이고 있다. 개인적으로 2020년의 영화 수급이 추춤한 이유는 코로나로 인해서 영화계에 타격이 있는 것은 아닐까 생각이 된다. 코로나에 관련된 데이터가 있었다면 상관관계를 파악하여 성장율 제동에 영향이 있었는지 분석해볼 필요가 있다.

The graph below is a graph of Netflix's supply and demand volume of "Movie" and "TV Show" over the past five years. 
Netflix increased the supply and demand of content by a large margin in 2017 compared to 2016.
The growth rate of movie content was high, but the growth rate decreased slightly in 2020. 
"TV Show" is not growing much compared to movie content, but it is steadily increasing. 

Personally, I think the reason why the supply and demand of films decreased slightly in 2020 is because of the corona.
If there was data related to corona, it is necessary to identify the correlation and analyze whether it affected the growth rate braking.

In [ ]:
grouped = dataset_last_5.groupby(['year_added', 'type'])
grouped_size = grouped.size()
grouped_size.unstack().plot.bar()
print(grouped_size)

다음은 영화가 최근 5년동안 러닝타임 추세를 보기 위해서 분석하였다. 러닝타임이 사용자의 선호도와 관련있지 않을까 생각했지만 의미있는 변화를 파악하지 못했다.

Next, the movie was analyzed to see the trend of running time over the past five years. 
Running time doesn't seem to have much impact on users choosing movies.

In [ ]:
movie_last_5 = dataset_last_5[dataset_last_5['type'] == 'Movie']
movie_last_5.loc[:,'duration'] = movie_last_5.loc[:,'duration'].str.split(' ').str[0]
movie_last_5['duration'] = movie_last_5['duration'].astype('int')
g = sns.boxplot(x = 'year_added', y = 'duration', data = movie_last_5)

최근 5년간 'TV Show'의 시즌제 콘텐츠의 비율을 파악하기 위해 진행하였다. 시즌제 콘텐츠가 점점 증가하는 추세를 보였다.

In the past five years, we have identified the percentage of series content on 'TV Show'. Series content showed an increasing trend.

In [ ]:
tv_last_5 = dataset_last_5[dataset_last_5['type'] == 'TV Show']
tv_last_5.loc[:,'duration'] = tv_last_5.loc[:,'duration'].str.split(' ').str[0]
tv_last_5['duration'] = tv_last_5['duration'].astype('int')
tv_season = pd.DataFrame(columns = ['TV_serials', 'TV_series'])

for i in range(5) :
    x = len(tv_last_5[(tv_last_5['year_added'] == 2016+i) & (tv_last_5['duration'] == 1)])
    y = len(tv_last_5[(tv_last_5['year_added'] == 2016+i)]) - x
    tv_season.loc[i] = [x,y]

tv_season.index = ['2016', '2017', '2018', '2019', '2020']
print(tv_season)
tv_season.plot.bar(rot = 0)

다음은 넷플릭스 안에서 영화와 tv 프로그램의 장르비율을 파악하였다. 

영화와 tv 프로그램 모두 드라마 장르가 가장 많았다. 드라마 장르는 정서적인 주제와 현실적인 등장 인물의 성격 묘사를 바탕으로 표현하는 장르라고 한다. 드라마 장르 안에 로맨스나 액션등 다양한 세부 장르가 포함되기 때문에 가장 많았다. 드라마 장르가 포괄적이기 때문에 각 장르별로 본다면 코미디 장르가 영화, tv 프로그램 할 것 없이 두드러졌다. 

아래의 코딩 중에 movie_genre_count.drop(movie_genre_count.index[[0,1]])를 통해 2개의 인덱스를 제거하였는데, 이 이유는 index[0]은 '영화', index[1]은 '국제 영화' 등으로 구성되어 있었기 때문이다. 넷플릭스에 직접 들어가보면 사용자의 국가에 따라서 국내 영화와 국제 영화 카테고리가 나뉘어 추천하고 있는데, 해당 데이터를 수집할 때 수집한 인물의 국가에 따라서 정해져 나온것인지 모르겠다. 현재 분석에선 불필요한 데이터라고 판단하여 제거하게 되었다.


Next, we found out the genre ratio of movies and TV programs within Netflix.

Both movies and TV programs had the most drama genres. 
The drama genre is said to be based on emotional themes and realistic characteristics. 
It was the most common because the drama genre included various detailed genres such as romance and action. 
Except for the drama genre, both movies and TV shows had many comedy genres.


Two indexes were removed from the coding below through “movie_genre_count.drop(movie_genre_count.index[[0,1]])” because index[0] consisted of 'movies' and index[1] consisted of 'international movies'. 
If you go to Netflix, domestic and international film categories are divided and recommended according to the user's country, but it may have been determined according to the country of the person collected when collecting the data. 
The current analysis determined that it was unnecessary data, so it was removed.


In [ ]:
movie_listed_in = list(set(sum(movie_last_5['listed_in'].str.split(', '), [])))
counts = []
for genre in movie_listed_in :
    counts.append(len(movie_last_5[movie_last_5['listed_in'].str.contains(genre)]))
    
movie_genre_count = pd.DataFrame([movie_listed_in, counts]).transpose()
movie_genre_count.columns = ['genre', 'count']
movie_genre_count = movie_genre_count.sort_values(by = ['count'], axis = 0, ascending = False)
movie_genre_count = movie_genre_count.drop(movie_genre_count.index[[0,1]])
print(movie_genre_count)
x = movie_genre_count
g = sns.barplot(x = x['count'], y = x['genre'])

In [ ]:
tv_listed_in = list(set(sum(tv_last_5['listed_in'].str.split(', '), [])))
counts = []
for genre in tv_listed_in :
    counts.append(len(tv_last_5[tv_last_5['listed_in'].str.contains(genre)]))
    
tv_genre_count = pd.DataFrame([tv_listed_in, counts]).transpose()
tv_genre_count.columns = ['genre', 'count']
tv_genre_count = tv_genre_count.sort_values(by = ['count'], axis = 0, ascending = False)
tv_genre_count = tv_genre_count.drop(tv_genre_count.index[[0,1]])
print(tv_genre_count)
x = tv_genre_count
g = sns.barplot(x = x['count'], y = x['genre'])